<a href="https://colab.research.google.com/github/Seyusumin/Mapa-interactivo-cultivos-en-mexico/blob/main/Mapa_interactivo_cultivos_en_mexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from sklearn.neighbors import NearestNeighbors

import geopy.distance
import folium
from folium.plugins import MarkerCluster

import warnings
from tqdm import tqdm

sns.set_style('darkgrid')
warnings.filterwarnings('ignore')

In [40]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [41]:
path = '/content/Cierre_agr_mun_2021.csv'
df =  pd.read_csv(path, sep=',', index_col=0, encoding='latin-1') 

In [42]:
# Exploracion y limpieza de datos
df.columns

Index(['Idestado', 'Nomestado', 'Idddr', 'Nomddr', 'Idcader', 'Nomcader',
       'Idmunicipio', 'Nommunicipio', 'Idciclo', 'Nomcicloproductivo',
       'Idmodalidad', 'Nommodalidad', 'Idunidadmedida', 'Nomunidad',
       'Idcultivo', 'Nomcultivo', 'Sembrada', 'Cosechada', 'Siniestrada',
       'Volumenproduccion', 'Rendimiento', 'Preciomediorural',
       'Valorproduccion'],
      dtype='object')

In [43]:
df.shape

(28741, 23)

In [44]:
df.describe()

,Idestado,Idddr,Idcader,Idmunicipio,Idciclo,Idmodalidad,Idunidadmedida,Idcultivo,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Preciomediorural,Valorproduccion
count,28741.000000,28741.000000,28741.000000,28741.000000,28741.000000,28741.000000,28741.000000,2.874100e+04,28741.000000,28741.000000,2.874100e+04,28741.000000,28741.000000,2.874100e+04
mean,18.404474,99.162033,2.676281,85.953203,2.023625,1.448558,200201.036707,7.155142e+06,599.478674,18.847497,8.333445e+03,109.297670,7779.538990,2.410593e+07
std,7.442896,47.392978,1.952962,116.360437,0.733030,0.497355,0.343560,1.234208e+06,2436.476882,590.117121,6.478327e+04,2655.112339,10482.510111,1.180360e+08
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200201.000000,5.050000e+06,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,13.000000,64.000000,1.000000,17.000000,1.000000,1.000000,200201.000000,6.110000e+06,10.400000,0.000000,7.080000e+01,2.150000,3600.000000,3.920400e+05
50%,20.000000,104.000000,2.000000,43.000000,2.000000,1.000000,200201.000000,7.470000e+06,43.000000,0.000000,3.825000e+02,8.400000,5135.630000,1.671346e+06
75%,24.000000,127.000000,3.000000,100.000000,3.000000,2.000000,200201.000000,7.960000e+06,245.000000,0.000000,2.099240e+03,21.400000,8518.730000,8.294000e+06
max,32.000000,193.000000,19.000000,570.000000,3.000000,2.000000,200206.000000,9.330000e+06,114365.000000,54073.000000,4.656950e+06,159000.000000,174754.550000,3.912576e+09


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28741 entries, 2021 to 2021
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Idestado            28741 non-null  int64  
 1   Nomestado           28741 non-null  object 
 2   Idddr               28741 non-null  int64  
 3   Nomddr              28741 non-null  object 
 4   Idcader             28741 non-null  int64  
 5   Nomcader            28741 non-null  object 
 6   Idmunicipio         28741 non-null  int64  
 7   Nommunicipio        28741 non-null  object 
 8   Idciclo             28741 non-null  int64  
 9   Nomcicloproductivo  28741 non-null  object 
 10  Idmodalidad         28741 non-null  int64  
 11  Nommodalidad        28741 non-null  object 
 12  Idunidadmedida      28741 non-null  int64  
 13  Nomunidad           28741 non-null  object 
 14  Idcultivo           28741 non-null  int64  
 15  Nomcultivo          28741 non-null  object 
 16  Se

In [46]:
df.head()

,Idestado,Nomestado,Idddr,Nomddr,Idcader,Nomcader,Idmunicipio,Nommunicipio,Idciclo,Nomcicloproductivo,...,Nomunidad,Idcultivo,Nomcultivo,Sembrada,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Preciomediorural,Valorproduccion
Anio,,,,,,,,,,,,,,,,,,,,,
2021,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,...,Tonelada,5490000,Avena forrajera en verde,1172,1172.0,0.0,31632.28,26.99,562.77,17801698.22
2021,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Otoño-Invierno,...,Tonelada,5670000,Brócoli,35,30.0,5.0,631.80,21.06,3921.85,2477824.83
2021,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,2,Primavera-Verano,...,Tonelada,5490000,Avena forrajera en verde,154,154.0,0.0,4521.44,29.36,583.25,2637129.88
2021,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,2,Primavera-Verano,...,Tonelada,5670000,Brócoli,25,25.0,0.0,376.25,15.05,4053.50,1525129.38
2021,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,2,Primavera-Verano,...,Tonelada,6820000,Fresa,5,5.0,0.0,205.00,41.00,17946.56,3679044.80


In [47]:
df = df.drop(["Idestado", "Idddr", "Nomddr","Idmodalidad", "Nommunicipio",
              "Nomcicloproductivo", "Idunidadmedida", "Nomunidad",
              "Nommodalidad", "Idcader", "Nomcader", "Idmunicipio", "Idciclo",
              "Idcultivo", "Siniestrada"], axis=1)
df.head(5)

,Nomestado,Nomcultivo,Sembrada,Cosechada,Volumenproduccion,Rendimiento,Preciomediorural,Valorproduccion
Anio,,,,,,,,
2021,Aguascalientes,Avena forrajera en verde,1172,1172.0,31632.28,26.99,562.77,17801698.22
2021,Aguascalientes,Brócoli,35,30.0,631.80,21.06,3921.85,2477824.83
2021,Aguascalientes,Avena forrajera en verde,154,154.0,4521.44,29.36,583.25,2637129.88
2021,Aguascalientes,Brócoli,25,25.0,376.25,15.05,4053.50,1525129.38
2021,Aguascalientes,Fresa,5,5.0,205.00,41.00,17946.56,3679044.80


In [48]:
df.head(48)

,Nomestado,Nomcultivo,Sembrada,Cosechada,Volumenproduccion,Rendimiento,Preciomediorural,Valorproduccion
Anio,,,,,,,,
2021,Aguascalientes,Avena forrajera en verde,1172,1172.00,31632.28,26.99,562.77,1.780170e+07
2021,Aguascalientes,Brócoli,35,30.00,631.80,21.06,3921.85,2.477825e+06
2021,Aguascalientes,Avena forrajera en verde,154,154.00,4521.44,29.36,583.25,2.637130e+06
2021,Aguascalientes,Brócoli,25,25.00,376.25,15.05,4053.50,1.525129e+06
2021,Aguascalientes,Fresa,5,5.00,205.00,41.00,17946.56,3.679045e+06
2021,Aguascalientes,Maíz forrajero en verde,3811,3811.00,244437.54,64.14,722.68,1.766501e+08
2021,Aguascalientes,Maíz grano,615,615.00,4378.80,7.12,3397.29,1.487605e+07
2021,Aguascalientes,Sorgo forrajero en verde,145,145.00,8022.85,55.33,577.48,4.633035e+06
2021,Aguascalientes,Tomate rojo (jitomate),19,19.00,2498.75,131.51,23634.88,5.905765e+07


In [53]:
df_maíz = df[df["Nomcultivo"] == "Maíz"]
df_maíz_grano = df_maíz[df_maíz['Nomcultivo'] == "Maíz grano"]

In [54]:
#coordenadas de la republica Mexicana
mexico_coords = [23.6345, -102.5528]

In [55]:
#crear mapa
m = folium.Map(location=mexico_coords, zoom_start=5)

In [58]:
#agregar marcadores para cada estado
for _, row in df_maíz_grano.iterrows():
    estado = row['Nomestado']
    coords = [row['Latitud'], row['Longitud']]
    produccion = row['Volumenproducción (Toneladas)']
    folium.Marker(coords, tooltip=estado, popup=f"{estado}: {produccion:.2f} Toneladas").add_to(m)
#mostrar mapa
m